**13. Using the Boston data set, fit classification models in order to predict
whether a given suburb has a crime rate above or below the median.
Explore logistic regression, LDA, and KNN models using various sub-
sets of the predictors. Describe your findings.**

In [1]:
# To support both python 2 and python 3
# 让这份笔记同步支持 python 2 和 python 3
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#使用statsmodels便于统计分析数据
import statsmodels.api as sm
import statsmodels.formula.api as smf
#使用sklearn便于预测（机器学习）
from sklearn import linear_model
#使用patsy便于生成模型
import patsy
%matplotlib inline

In [2]:
Boston = pd.read_csv("data/Boston.csv")
Boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
Boston['crim'].median()

0.25651

In [4]:
Boston['crim01'] = (Boston['crim'] >= Boston['crim'].median()) * 1
Boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,crim01
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0,0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6,0
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7,0
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4,0
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2,0


In [5]:
Boston.sort_values('crim',ascending=False).head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,crim01
380,88.9762,0.0,18.1,0,0.671,6.968,91.9,1.4165,24,666.0,20.2,396.90,17.21,10.4,1
418,73.5341,0.0,18.1,0,0.679,5.957,100.0,1.8026,24,666.0,20.2,16.45,20.62,8.8,1
405,67.9208,0.0,18.1,0,0.693,5.683,100.0,1.4254,24,666.0,20.2,384.97,22.98,5.0,1
410,51.1358,0.0,18.1,0,0.597,5.757,100.0,1.4130,24,666.0,20.2,2.60,10.11,15.0,1
414,45.7461,0.0,18.1,0,0.693,4.519,100.0,1.6582,24,666.0,20.2,88.27,36.98,7.0,1


In [6]:
Boston.corr()['crim01'].sort_values()

dis       -0.616342
zn        -0.436151
black     -0.351211
medv      -0.263017
rm        -0.156372
chas       0.070097
ptratio    0.253568
crim       0.409395
lstat      0.453263
indus      0.603260
tax        0.608741
age        0.613940
rad        0.619786
nox        0.723235
crim01     1.000000
Name: crim01, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(Boston,test_size=0.2, random_state=427)

In [8]:
# logistic regression
from sklearn.linear_model import LogisticRegression
dmatrices_model = '0'
for x_factor in Boston.columns:
    dmatrices_model = dmatrices_model + '+' + x_factor
y_train,X_train = patsy.dmatrices('crim01 ~ {} - crim01 - crim'.format(dmatrices_model),data = train_set)
y_test,X_test = patsy.dmatrices('crim01 ~ {} - crim01 - crim'.format(dmatrices_model),data = test_set)
log_reg = LogisticRegression(penalty = 'none',max_iter = 10000)
log_reg.fit(X_train, np.c_[y_train].ravel())
log_reg.score(X_test,np.c_[y_test].ravel())

0.8823529411764706

In [9]:
# LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, np.c_[y_train].ravel())
lda.score(X_test,np.c_[y_test].ravel())

0.803921568627451

In [10]:
# QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train, np.c_[y_train].ravel())
qda.score(X_test,np.c_[y_test].ravel())

0.8921568627450981

In [11]:
#方法1，使用k=1,10,100比较
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(427)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, np.c_[y_train].ravel())
print('k=1',knn.score(X_test,np.c_[y_test].ravel()))
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, np.c_[y_train].ravel())
print('k=10',knn.score(X_test,np.c_[y_test].ravel()))
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, np.c_[y_train].ravel())
print('k=100',knn.score(X_test,np.c_[y_test].ravel()))

k=1 0.9117647058823529
k=10 0.8823529411764706
k=100 0.7647058823529411


In [12]:
#方法2，使用用GridSearchCV搜寻最好的k参数
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
param_grid = [
{'n_neighbors': list(range(1,200))},
]
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, np.c_[y_train].ravel())
grid_search.best_params_

{'n_neighbors': 1}

**如果考虑所有变量纳入模型，则knn的方案中k=1预测精度最好**

In [13]:
#对比各种模型下的比较
factors = list(Boston.columns)
factors.remove('crim')
factors.remove('crim01')
response = 'crim01'
def get_models(factors,response,dic_or_list = 1,intercept = 0):
    dmatrices_models_dic = {}
    dmatrices_models_list = []
    f_len = len(factors)
    for model_len in range(f_len):
        dmatrices_models_dic[model_len] = []
        if model_len == 0:
            for factor_num in range(f_len):
                if intercept == 1:
                    dmatrices_models_dic[model_len].append(response + '~' + factors[factor_num])
                    dmatrices_models_list.append(response + '~' + factors[factor_num])
                else:
                    dmatrices_models_dic[model_len].append(response + '~ 0 +' + factors[factor_num])
                    dmatrices_models_list.append(response + '~ 0 +' + factors[factor_num])
        else:
            m_len = len(dmatrices_models_dic[model_len - 1])
            for m_num in range(m_len):
                for factor_num in range(f_len - 1,model_len - 1,-1):
                    if factors[factor_num] not in dmatrices_models_dic[model_len - 1][m_num]:
                        dmatrices_models_dic[model_len].append(dmatrices_models_dic[model_len - 1][m_num] + '+' + factors[factor_num])
                        dmatrices_models_list.append(dmatrices_models_dic[model_len - 1][m_num] + '+' + factors[factor_num])
                    else:
                        break 
    if dic_or_list == 0:
        dmatrices_models = dmatrices_models_dic
    else:
        dmatrices_models = dmatrices_models_list
    return dmatrices_models
dmatrices_models = get_models(factors,response)

In [14]:
#对logic得分的比较
score_models = {}
for dmatrices_model in dmatrices_models:
    y_train,X_train = patsy.dmatrices(dmatrices_model,data = train_set,return_type="dataframe")
    y_test,X_test = patsy.dmatrices(dmatrices_model,data = test_set,return_type="dataframe")
    log_reg.fit(X_train, np.c_[y_train].ravel())
    score_models[dmatrices_model] = log_reg.score(X_test, np.c_[y_test].ravel())
score_models_data = pd.DataFrame(score_models,index = ['score']).T
score_models_data.sort_values('score',ascending=False).iloc[:20]

,score
crim01~ 0 +zn+indus+nox+age+dis+rad+ptratio+black,0.921569
crim01~ 0 +zn+indus+nox+age+dis+rad+black,0.921569
crim01~ 0 +zn+indus+nox+dis+rad+ptratio+black,0.911765
crim01~ 0 +indus+nox+rad+ptratio+black,0.911765
crim01~ 0 +indus+nox+age+rad+black,0.911765
crim01~ 0 +zn+indus+nox+age+dis+rad+black+lstat+medv,0.911765
crim01~ 0 +indus+nox+age+rad+ptratio,0.911765
crim01~ 0 +zn+indus+nox+rad+ptratio+black+medv,0.911765
crim01~ 0 +zn+indus+chas+nox+age+dis+rad,0.911765
crim01~ 0 +zn+indus+nox+rad+ptratio+black+lstat,0.911765


In [15]:
#对lda得分的比较
score_models = {}
for dmatrices_model in dmatrices_models:
    y_train,X_train = patsy.dmatrices(dmatrices_model,data = train_set,return_type="dataframe")
    y_test,X_test = patsy.dmatrices(dmatrices_model,data = test_set,return_type="dataframe")
    lda.fit(X_train, np.c_[y_train].ravel())
    score_models[dmatrices_model] = lda.score(X_test, np.c_[y_test].ravel())
score_models_data = pd.DataFrame(score_models,index = ['score']).T
score_models_data.sort_values('score',ascending=False).iloc[:20]

,score
crim01~ 0 +zn+chas+rm+age+rad+tax+black,0.872549
crim01~ 0 +rm+age+tax+medv,0.872549
crim01~ 0 +zn+chas+rm+dis+rad+ptratio+lstat,0.872549
crim01~ 0 +chas+rm+age+tax+medv,0.872549
crim01~ 0 +zn+chas+rm+age+rad+tax+black+lstat,0.872549
crim01~ 0 +chas+rm+age+rad+tax+black,0.872549
crim01~ 0 +chas+dis+rad+medv,0.872549
crim01~ 0 +chas+rm+age+rad+tax+black+lstat,0.872549
crim01~ 0 +rm+age+tax+black+medv,0.862745
crim01~ 0 +chas+age+tax+ptratio+medv,0.862745


In [16]:
#对qda得分的比较
score_models = {}
for dmatrices_model in dmatrices_models:
    y_train,X_train = patsy.dmatrices(dmatrices_model,data = train_set,return_type="dataframe")
    y_test,X_test = patsy.dmatrices(dmatrices_model,data = test_set,return_type="dataframe")
    qda.fit(X_train, np.c_[y_train].ravel())
    score_models[dmatrices_model] = qda.score(X_test, np.c_[y_test].ravel())
score_models_data = pd.DataFrame(score_models,index = ['score']).T
score_models_data.sort_values('score',ascending=False).iloc[:20]

,score
crim01~ 0 +zn+indus+chas+nox+dis+rad+tax+ptratio+lstat,0.950980
crim01~ 0 +zn+nox+dis+rad+tax+ptratio+lstat,0.950980
crim01~ 0 +zn+indus+chas+nox+dis+rad+tax+ptratio+lstat+medv,0.950980
crim01~ 0 +zn+indus+nox+rm+dis+rad+tax+ptratio+lstat,0.941176
crim01~ 0 +indus+age+dis+rad+tax+ptratio+lstat+medv,0.941176
crim01~ 0 +zn+indus+nox+dis+rad+tax+ptratio+lstat,0.941176
crim01~ 0 +zn+indus+nox+dis+rad+tax+ptratio+lstat+medv,0.941176
crim01~ 0 +indus+rm+age+dis+rad+tax+ptratio+lstat,0.941176
crim01~ 0 +zn+indus+chas+rm+dis+rad+tax+ptratio+lstat,0.941176
crim01~ 0 +zn+indus+dis+rad+tax+ptratio+lstat+medv,0.931373


In [17]:
#对knn得分(K=1)的比较
score_models = {}
knn = KNeighborsClassifier(n_neighbors=1)
for dmatrices_model in dmatrices_models:
    y_train,X_train = patsy.dmatrices(dmatrices_model,data = train_set,return_type="dataframe")
    y_test,X_test = patsy.dmatrices(dmatrices_model,data = test_set,return_type="dataframe")
    knn.fit(X_train, np.c_[y_train].ravel())
    score_models[dmatrices_model] = knn.score(X_test, np.c_[y_test].ravel())
score_models_data = pd.DataFrame(score_models,index = ['score']).T
score_models_data.sort_values('score',ascending=False).iloc[:20]

,score
crim01~ 0 +nox+rm+dis+rad+ptratio,0.980392
crim01~ 0 +rm+dis+rad+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+rad+ptratio,0.980392
crim01~ 0 +zn+indus+rm+dis+rad,0.980392
crim01~ 0 +zn+indus+rm+dis+tax,0.980392
crim01~ 0 +zn+indus+rm+dis+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+tax+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+tax,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+rad,0.980392


In [18]:
score_models_data.sort_values('score',ascending=False).iloc[:35]

,score
crim01~ 0 +nox+rm+dis+rad+ptratio,0.980392
crim01~ 0 +rm+dis+rad+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+rad+ptratio,0.980392
crim01~ 0 +zn+indus+rm+dis+rad,0.980392
crim01~ 0 +zn+indus+rm+dis+tax,0.980392
crim01~ 0 +zn+indus+rm+dis+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+tax+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+ptratio,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+tax,0.980392
crim01~ 0 +zn+indus+nox+rm+dis+rad,0.980392
